## Share People Hub — coleta direta via API InHire

Busca vagas diretamente na API pública do InHire para o tenant `sharepeoplehub` (mesmo domínio `sharepeoplehub.inhire.app`). Filtra por palavras-chave (ex.: Java) e salva um CSV.

> Requer internet. Se a requisição falhar, revise proxy/VPN ou teste o endpoint manualmente.

In [5]:
print("Coletando vagas diretamente do InHire (tenant: sharepeoplehub)...")

#!pip install requests pandas > /dev/null

import requests
import pandas as pd

TENANT = "sharepeoplehub"
API_URL = "https://api.inhire.app/job-posts/public/pages"
KEYWORDS = ["Java", "Backend"]  # ajuste as palavras-chave

headers = {"X-Tenant": TENANT, "User-Agent": "Mozilla/5.0 (scraper)"}
resp = requests.get(API_URL, headers=headers, timeout=30)
resp.raise_for_status()
data = resp.json()

jobs = data.get("jobsPage", [])
rows = []
for job in jobs:
    # Alguns retornos vêm como objetos planos; outros trazem a vaga dentro de job/jobInfo
    j = job.get("job") if isinstance(job, dict) and "job" in job else job
    title = j.get("displayName") or j.get("name") or j.get("title")
    desc = j.get("description") or ""
    city = j.get("locationCity") or j.get("city") or ""
    state = j.get("locationState") or j.get("state") or ""
    link_id = j.get("id") or j.get("jobId") or j.get("slug") or ""
    link = f"https://{TENANT}.inhire.app/vagas/{link_id}" if link_id else ""
    rows.append({
        "titulo": title,
        "descricao": desc,
        "cidade": city,
        "estado": state,
        "area": j.get("area"),
        "senioridade": j.get("seniority"),
        "contrato": j.get("contractType"),
        "id": link_id,
        "link": link,
    })

df = pd.DataFrame(rows)
print(f"Total de vagas retornadas: {len(df)}")

# filtro opcional por palavra-chave e remoto
if not df.empty:
    mask_kw = df["titulo"].fillna("").str.contains("|".join(KEYWORDS), case=False, na=False)
    mask_desc = df["descricao"].fillna("").str.contains("|".join(KEYWORDS), case=False, na=False)
    mask_remote_full = df[["cidade", "descricao"]].fillna("").apply(
        lambda s: s.str.contains("remoto", case=False, na=False)
    ).any(axis=1)

    df_filtrado = df[mask_kw | mask_desc]
    # reindex to avoid boolean misalignment warning
    df_java_remoto = df_filtrado[mask_remote_full.loc[df_filtrado.index]]

    print(
        f"Filtradas por keywords ({KEYWORDS}): {len(df_filtrado)} | Java remoto: {len(df_java_remoto)}"
    )
else:
    df_filtrado = df_java_remoto = df

output_all = "vagas_sharepeoplehub_todas.csv"
df.to_csv(output_all, index=False)
print(f"CSV completo salvo em {output_all}")

output_filt = "vagas_sharepeoplehub_java_remoto.csv"
df_java_remoto.to_csv(output_filt, index=False)
print(f"CSV filtrado salvo em {output_filt}")

df_java_remoto.head()


Coletando vagas diretamente do InHire (tenant: sharepeoplehub)...
Total de vagas retornadas: 58
Filtradas por keywords (['Java', 'Backend']): 0 | Java remoto: 0
CSV completo salvo em vagas_sharepeoplehub_todas.csv
CSV filtrado salvo em vagas_sharepeoplehub_java_remoto.csv


,titulo,descricao,cidade,estado,area,senioridade,contrato,id,link
